In [1]:
import numpy as np
import pandas as pd
import theano
import theano.tensor as T

import lasagne

theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [32]:
path="data/"
data=pd.read_csv(path+"input_train.csv", index_col="ID")
X_ACC=data.filter(regex= "ACC*")
X_EEG=data.filter(regex= "EEG*")
labels=pd.read_csv(path+"challenge_output_data_training_file_sleep_stages_classification.csv", sep=";")

In [33]:
data_test=pd.read_csv(path+"input_test.csv", index_col="ID")
X_ACC_TEST=data_test.filter(regex= "ACC*")
X_EEG_TEST=data_test.filter(regex= "EEG*")

In [34]:
from sklearn.cross_validation import train_test_split
def train_val_test_split(X):    
    X_train, X_test = train_test_split(X, test_size=0.10, random_state=42)
    X_train, X_val = train_test_split(X_train, test_size=0.10, random_state=42)
    return X_train, X_val, X_test

In [35]:
X_ACC_train, X_ACC_val, X_ACC_test=train_val_test_split(X_ACC)
X_EEG_train, X_EEG_val, X_EEG_test=train_val_test_split(X_EEG)
y_train, y_val, y_test=train_val_test_split(labels["TARGET"])

In [ ]:
X_ACC_TEST.shape, X_ACC.shape

In [22]:
X_ACC_TEST.shape

(30458, 450)

In [36]:
from sklearn.preprocessing import StandardScaler

scaler_ACC_train = StandardScaler().fit(X_ACC_train)
scaler_EEG_train= StandardScaler().fit(X_EEG_train)
scaler_ACC_predict = StandardScaler().fit(X_ACC)
scaler_EEG_predict= StandardScaler().fit(X_EEG)

X_ACC_train=scaler_ACC_train.transform(X_ACC_train)
X_ACC_val=scaler_ACC_train.transform(X_ACC_val)
X_ACC_test=scaler_ACC_train.transform(X_ACC_test)

X_EEG_train=scaler_EEG_train.transform(X_EEG_train)
X_EEG_val=scaler_EEG_train.transform(X_EEG_val)
X_EEG_test=scaler_EEG_train.transform(X_EEG_test)

X_ACC=scaler_ACC_predict.transform(X_ACC)
X_EEG=scaler_EEG_predict.transform(X_EEG)

X_ACC_TEST=scaler_ACC_predict.transform(X_ACC_TEST)
X_EEG_TEST=scaler_EEG_predict.transform(X_EEG_TEST)

In [37]:
def reshape(X, n_channels):
    return X.reshape(X.shape[0],n_channels,X.shape[1]/n_channels)

In [30]:
%pdb

Automatic pdb calling has been turned ON


In [38]:
X_EEG_train=reshape(X_EEG_train,1)
X_EEG_val=reshape(X_EEG_val,1)
X_EEG_test=reshape(X_EEG_test,1)
X_EEG=reshape(X_EEG,1)
X_EEG_TEST=reshape(X_EEG_TEST,1)

X_ACC_train=reshape(X_ACC_train,3)
X_ACC_val=reshape(X_ACC_val,3)
X_ACC_test=reshape(X_ACC_test,3)
X_ACC=reshape(X_ACC,3)
X_ACC_TEST=reshape(X_ACC_TEST,3)

In [39]:
# Prepare Theano variables for inputs and targets
input_var_EEG = T.tensor3('inputs')
input_var_ACC = T.tensor3('inputs')
target_var = T.ivector('targets')
# Create neural network model

In [82]:
import time
from sklearn.base import BaseEstimator
def iterate_minibatches(input_1, input_2, targets, batchsize, shuffle=False):
    assert len(input_1) == len(targets)
    assert len(input_2) == len(targets)
    if shuffle:
        indices = np.arange(len(input_1))
        np.random.shuffle(indices)
    for start_idx in range(0, len(input_1) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield input_1[excerpt], input_2[excerpt], targets[excerpt]

"""Default variables"""
num_epochs = 1
EEG_shape_1=X_EEG_train.shape[1]
EEG_shape_2=X_EEG_train.shape[2]
ACC_shape_1=X_ACC_train.shape[1]
ACC_shape_2=X_ACC_train.shape[2]
n_filters_EEG_1=32
#n_filters_EEG_2=
n_filters_ACC_1=16
#n_filters_ACC_2=
filter_size_EEG_1=100
#filter_size_EEG_2=
filter_size_ACC_1=8
#filter_size_ACC_2=
pool_size_EEG=25
pool_size_ACC=2
drop_out=0.7
init_learning_rate=0.01
num_units_EEG=1000
num_units_ACC=100
num_units_pool=1000
num_classes=5

class CNN(BaseEstimator):
    def __init__(
        self,
        input_var_EEG, 
        input_var_ACC, 
        target_var,
        num_epochs
    ):
        #setting variables
        self.input_var_EEG=input_var_EEG
        self.input_var_ACC=input_var_ACC
        self.target_var=target_var    
        self.num_epochs=num_epochs
        
        #The network
        #Two initialisation layers
        #One for EEG
        self.l_in1 = lasagne.layers.InputLayer(
            shape=(None,EEG_shape_1, EEG_shape_2),
            input_var=self.input_var_EEG
        )
        #One for ACC
        self.l_in2 = lasagne.layers.InputLayer(
            shape=(None,ACC_shape_1, ACC_shape_2),
            input_var=self.input_var_ACC
        )
        #Two convolutional layers to treat each signal separatedly
        self.l_conv1_1 = lasagne.layers.Conv1DLayer(
            self.l_in1, 
            num_filters=n_filters_EEG_1, 
            filter_size=filter_size_EEG_1,
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotNormal()
        )
        self.l_conv2_1 = lasagne.layers.Conv1DLayer(
            self.l_in2, 
            num_filters=n_filters_ACC_1, 
            filter_size=filter_size_ACC_1,
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotNormal()
        )
        #And again another 2 layers
        """l_conv1_2 = lasagne.layers.Conv1DLayer(
            l_conv1_1, num_filters=16, filter_size=50,
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotUniform())
        l_conv2_2 = lasagne.layers.Conv1DLayer(
            l_conv2_1, num_filters=16, filter_size=5,
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotUniform())
        """
        #And two pooling layers
        """l_pool1 = lasagne.layers.MaxPool1DLayer(l_conv1_2, pool_size=25)
        l_pool2 = lasagne.layers.MaxPool1DLayer(l_conv2_2, pool_size=2)
        """
        self.l_pool1 = lasagne.layers.MaxPool1DLayer(
            self.l_conv1_1, 
            pool_size=pool_size_EEG
        )
        self.l_pool2 = lasagne.layers.MaxPool1DLayer(
            self.l_conv2_1, 
            pool_size=pool_size_ACC
        )    
        #Two dense layers for each signal
        self.l_dense_1_1 = lasagne.layers.DenseLayer(
            lasagne.layers.dropout(
                self.l_pool1, 
                p=drop_out
            ),
            num_units=num_units_EEG,
            nonlinearity=lasagne.nonlinearities.rectify
        )
        self.l_dense_2_1 = lasagne.layers.DenseLayer(
                lasagne.layers.dropout(
                self.l_pool2, 
                p=drop_out
            ),
            num_units=num_units_ACC,
            nonlinearity=lasagne.nonlinearities.rectify
        )
        #use neurons from both signals to predict
        #concatenate neurons
        self.l_concat = lasagne.layers.ConcatLayer(
            [
                self.l_dense_1_1, 
                self.l_dense_2_1
            ], 
            axis=1
        )
        #dense layer with all neurons
        self.l_dense_2 = lasagne.layers.DenseLayer(
                lasagne.layers.dropout(
                self.l_concat, 
                p=drop_out
            ),
            num_units=num_units_pool,
            nonlinearity=lasagne.nonlinearities.rectify
        )
        #Output layer
        self.l_output = lasagne.layers.DenseLayer(
            lasagne.layers.dropout(
                self.l_dense_2, 
                p=drop_out
            ),
            num_units=num_classes,
            nonlinearity=lasagne.nonlinearities.softmax
        )
        self.prediction = lasagne.layers.get_output(self.l_output)
        self.loss = lasagne.objectives.multiclass_hinge_loss(self.prediction, target_var)
        self.loss = self.loss.mean()
        self.params = lasagne.layers.get_all_params(self.l_output, trainable=True)
        self.updates = lasagne.updates.adadelta(
            self.loss, 
            self.params, 
            learning_rate=init_learning_rate
        )
        self.test_prediction = lasagne.layers.get_output(self.l_output, deterministic=True)
        self.test_loss = lasagne.objectives.multiclass_hinge_loss(self.test_prediction,
                                                                target_var)
        self.test_loss = self.test_loss.mean()
        self.test_acc = T.mean(T.eq(T.argmax(self.test_prediction, axis=1), self.target_var),
                  dtype=theano.config.floatX)
        self.train_fn =theano.function(
            [self.input_var_EEG, self.input_var_ACC, self.target_var], 
            self.loss, 
            updates=self.updates
        )
        self.val_fn = theano.function(
            [self.input_var_EEG, 
             self.input_var_ACC,
             self.target_var
            ], 
            [self.test_loss, self.test_acc])
    def fit(self,X_EEG_train,X_ACC_train,X_EEG_val,X_ACC_val,y_train,y_val):

        for epoch in range(self.num_epochs):
            # In each epoch, we do a full pass over the training data:
            train_err = 0
            train_batches = 0
            start_time = time.time()
            for batch in iterate_minibatches(X_EEG_train, X_ACC_train, y_train, 64, shuffle=True):
                input_1, input_2, targets = batch
                targets = targets.astype(np.int32)
                train_err += self.train_fn(input_1, input_2, targets)
                train_batches += 1
                if train_batches % 100 == 0:
                    print "epoch {}, train batches{}".format(epoch,train_batches) 

            # And a full pass over the validation data:
            val_err = 0
            val_acc = 0
            val_batches = 0
            for batch in iterate_minibatches(X_EEG_val, X_ACC_val, y_val, 128, shuffle=False):
                input_1, input_2, targets = batch
                targets = targets.astype(np.int32)
                err, acc = self.val_fn(input_1, input_2, targets)
                val_err += err
                val_acc += acc
                val_batches += 1

            # Then we print the results for this epoch:
            print("Epoch {} of {} took {:.3f}s".format(
                epoch + 1, num_epochs, time.time() - start_time))
            print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
            print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
            print("  validation accuracy:\t\t{:.2f} %".format(
                val_acc / val_batches * 100))
        return self
        
    def predict_proba(self,X_EEG_test, X_ACC_test):
        net_output=lasagne.layers.get_output(self.l_output, deterministic=True)
        prediction_fn = theano.function([input_var_EEG, input_var_ACC], [net_output])
        prediction =  prediction_fn(X_EEG_test, X_ACC_test)
        return prediction[0]

    def predict(self,X_EEG_test, X_ACC_test):
         return np.argmax(self.predict_proba(X_EEG_test, X_ACC_test), axis=1)
        

In [83]:
cnn=CNN(input_var_EEG, 
        input_var_ACC, 
        target_var,
        num_epochs
       )

In [84]:
cnn.fit(X_EEG_train, X_ACC_train, X_EEG_val, X_ACC_val, y_train, y_val)

epoch 0, train batches100
epoch 0, train batches200
epoch 0, train batches300
Epoch 1 of 1 took 261.688s
  training loss:		1.064237
  validation loss:		1.007201
  validation accuracy:		49.67 %


CNN(input_var_ACC=inputs, input_var_EEG=inputs, num_epochs=1,
  target_var=targets)

In [85]:
cnn.predict_proba(X_EEG_test, X_ACC_test)

array([[ 0.14021251,  0.11619791,  0.35786882,  0.2419734 ,  0.14374737],
       [ 0.11177684,  0.10079557,  0.48230405,  0.16996395,  0.1351596 ],
       [ 0.121304  ,  0.11671714,  0.39195205,  0.23015233,  0.13987448],
       ..., 
       [ 0.0364896 ,  0.03863931,  0.53840717,  0.34013839,  0.04632554],
       [ 0.09384441,  0.09309949,  0.47041411,  0.22740766,  0.11523433],
       [ 0.002091  ,  0.00123766,  0.40399347,  0.59097655,  0.00170132]])

In [86]:
cnn.predict(X_EEG_test, X_ACC_test)

array([2, 2, 2, ..., 2, 2, 3])